# 1. Random permutations

## 1.a)

Is not fair because that would imply that all permutations should have equal probability. In this case, the choice of the second dependes on the first and Alice's positions sometimes is forced. So the distribution is not uniform.

## 1.b)

In [27]:
import itertools
import random

def generate_fair_schedule():
    people = ['Alice', 'Bob', 'Charly', 'Dave']
    valid_permutations = []
    
    # Generate all possible permutations
    for perm in itertools.permutations(people):
        if perm.index('Alice') in [0, 1]:  # Ensure Alice is in one of the first two spots
            valid_permutations.append(perm)
    
    # Select one permutation uniformly at random
    return random.choice(valid_permutations)

# Generate a fair schedule
schedule = generate_fair_schedule()
print("Bathroom schedule:", schedule)


Bathroom schedule: ('Alice', 'Bob', 'Charly', 'Dave')
